<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Assessment-OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Lingual Medical Assessment Classification

OpenAI Models over Remote calls

In [74]:
import sys
IN_COLAB = 'google.colab' in sys.modules # True if running in Google Colab.
IN_COLAB

True

In [75]:
if IN_COLAB:
    !pip install -q openai

In [32]:
OPENAI_API_KEY = '...'

assert OPENAI_API_KEY != '...', "Enter your OpenAI API Key"

In [76]:
from openai import OpenAI

client = OpenAI(
  api_key=OPENAI_API_KEY
)
models = client.models.list()

In [132]:
set([model.id for model in models])

{'babbage-002',
 'chatgpt-4o-latest',
 'dall-e-2',
 'dall-e-3',
 'davinci-002',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0125',
 'gpt-3.5-turbo-1106',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-instruct-0914',
 'gpt-4',
 'gpt-4-0125-preview',
 'gpt-4-0613',
 'gpt-4-1106-preview',
 'gpt-4-turbo',
 'gpt-4-turbo-2024-04-09',
 'gpt-4-turbo-preview',
 'gpt-4o',
 'gpt-4o-2024-05-13',
 'gpt-4o-2024-08-06',
 'gpt-4o-mini',
 'gpt-4o-mini-2024-07-18',
 'text-embedding-3-large',
 'text-embedding-3-small',
 'text-embedding-ada-002',
 'tts-1',
 'tts-1-1106',
 'tts-1-hd',
 'tts-1-hd-1106',
 'whisper-1'}

In [148]:
# kind = 'gpt-4-turbo'
# kind = 'gpt-4o'
# kind = 'gpt-4o-mini'
kind = 'gpt-3.5-turbo'

# lang = "de"
lang = "en"

In [149]:
positive_en = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [150]:
negative_en = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [151]:
positive_de = [
  "Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.",
  "Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.",
  "Alltagsrelevante Gebrauchsvorteile werden festgestellt.",
  "Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.",
  "Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor."
]

In [152]:
negative_de = [
  "Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.",
  "Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.",
  "Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.",
  "In der Gesamtschau der hier vorliegenden Informationen kann nicht erkannt werden, wie die Versorgung des Versicherten mit dem Produkt begründet werden könnte, noch kann die Unbedenklichkeit einer solchen Versorgung bestätigt werden.",
  "Eine ärztliche Begründung, warum im vorliegenden Fall ein nicht im Hilfsmittelverzeichnis gelistetes Produkt zum Einsatz kommen soll, wird nicht übermittelt."
]

In [153]:

if lang == "de":
  negative = negative_de
  positive = positive_de
else:
  negative = negative_en
  positive = positive_en



In [154]:
assessment = negative[0]
# assessment = positive[0]

In [155]:
if lang == "de":
  messages = [
    {"role": "system", "content": "Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell."},
    {"role": "user", "content": f'''
    Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

    # Assessment
    {assessment}

    '''}
  ]
else:
  messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
    {"role": "user", "content": f'''
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''}
]


In [156]:
completion = client.chat.completions.create(
  model=kind,
  messages=messages,
  temperature=0.0,
)
completion.choices[0].message.content

'No. The assessment does not provide specific findings to derive a positive or negative recommendation due to the lack of clarity in the diagnosis named as the basis for the regulation.'

In [157]:
def eval_assessment(assessment):
  if lang == "de":
    yes = "Ja"
    no = "Nein"
    messages = [
  {"role": "system", "content": "Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell."},
  {"role": "user", "content": f'''
  Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

  # Assessment
  {assessment}

  '''}
  ]
  else:
    yes = "Yes"
    no = "No"
    messages = [
      {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
      {"role": "user", "content": f'''
  What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

  # Assessment
  {assessment}

  '''}
  ]

  completion = client.chat.completions.create(
    model=kind,
    messages=messages,
    temperature=0.0,
  )
  result = completion.choices[0].message.content
  if result.startswith(yes):
    return "Positive", result
  elif result.startswith(no):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [158]:
%%time

negative_results = []
negative_explanations = []

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  negative_results.append(result)
  negative_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: No specific findings can be derived from the diagnosis currently named as the basis for the regulation.
Negative: No. The assessment indicates that no specific findings can be derived from the current diagnosis named as the basis for the regulation.
-----
Assessment: According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.
Negative: No. The insured has already been provided with the functional product requested according to its area of application, therefore no further recommendation is necessary.
-----
Assessment: A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.
Negative: No.

Justification: The assessment indicates that a medically comprehensible exp

## Positive

In [159]:
%%time

positive_results = []
positive_explanations = []

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  positive_results.append(result)
  positive_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.
Negative: No.

The assessment suggests that there is a need for compensation to ensure the basic need, indicating a potential positive recommendation for coverage under statutory health insurance.
-----
Assessment: The socio-medical prerequisites for the prescribed aid supply have been met.
Positive: Yes, a positive recommendation is given. The socio-medical prerequisites for the prescribed aid supply have been met, indicating that the individual meets the necessary criteria to receive the aid supply as prescribed.
-----
Assessment: Everyday relevant usage benefits have been determined.
Negative: No.

The assessment does not provide enough information to determine whether a positive or negative recommendation has been given. The statement only mentions that everyday relevant usage benefits have been determined, but does not specify the outcome of the assessment.
-----
Assessmen

In [160]:
import pandas as pd

df = pd.DataFrame({
    'assesment': negative + positive,
    'y_true': ['Negative'] * len(negative) + ['Positive'] * len(positive),
    'y_hat': negative_results + positive_results,
    'explanation': negative_explanations + positive_explanations
})
df

,assesment,y_true,y_hat,explanation
0,No specific findings can be derived from the d...,Negative,Negative,No. The assessment indicates that no specific ...
1,According to the service extracts from the hea...,Negative,Negative,No. The insured has already been provided with...
2,A medically comprehensible explanation as to w...,Negative,Negative,No.\n\nJustification: The assessment indicates...
3,From an overall view of the information availa...,Negative,Negative,No.\n\nJustification: Based on the assessment ...
4,A medical justification for why a product not ...,Negative,Negative,No.\n\nJustification: The assessment indicates...
5,"With the diagnosis named here, the need for co...",Positive,Negative,No.\n\nThe assessment suggests that there is a...
6,The socio-medical prerequisites for the prescr...,Positive,Positive,"Yes, a positive recommendation is given. The s..."
7,Everyday relevant usage benefits have been det...,Positive,Negative,No.\n\nThe assessment does not provide enough ...
8,Socio-medical indication for the aid is confir...,Positive,Positive,"Yes, a positive recommendation is given. The s..."
9,Contraindications have been excluded; there ar...,Positive,Positive,Yes.\n\nJustification: The assessment has conf...


In [161]:
df.to_excel(f'results_{kind}_{lang}.xlsx', index=False)

In [162]:
!ls -l

total 68
-rw-r--r-- 1 root root 6312 Aug 23 14:52 results_gpt-3.5-turbo_de.xlsx
-rw-r--r-- 1 root root 6161 Aug 23 14:53 results_gpt-3.5-turbo_en.xlsx
-rw-r--r-- 1 root root 6109 Aug 23 14:46 results_gpt-4o_de.xlsx
-rw-r--r-- 1 root root 6091 Aug 23 14:43 results_gpt-4o_en.xlsx
-rw-r--r-- 1 root root 6224 Aug 23 14:46 results_gpt-4o-mini_de.xlsx
-rw-r--r-- 1 root root 6112 Aug 23 14:41 results_gpt-4o-mini_en.xlsx
-rw-r--r-- 1 root root 6131 Aug 23 14:45 results_gpt-4-turbo_de.xlsx
-rw-r--r-- 1 root root 6332 Aug 23 14:44 results_gpt-4-turbo_en.xlsx
drwxr-xr-x 1 root root 4096 Aug 21 13:28 sample_data
